<a href="https://colab.research.google.com/github/stumblecow/playing-with-stuff/blob/main/playing_with_pandas_and_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import streamlit as st

# Set up the page
st.set_page_config(layout="wide")
st.title("📊 Delegate Projection Tool")

# 1. Load your data. PLEASE CHANGE THESE FILENAMES TO MATCH YOURS!
@st.cache_data
def load_data():
    # Replace 'chapters.csv' and 'caucuses.csv' with your actual file names
    chapters = pd.read_csv('chapters.csv')
    caucuses = pd.read_csv('caucuses.csv')
    return chapters, caucuses

base_chapters, base_caucuses = load_data()

# 2. Let the user set a global growth rate
global_growth = st.sidebar.slider("Global Growth Rate (%)", 0.0, 100.0, 10.0) / 100.0

# 3. Copy our base data so we can edit it without messing up the original
chapters_df = base_chapters.copy()
caucuses_df = base_caucuses.copy()

# 4. Add a new column for the user's growth rate override (starting with the global rate)
chapters_df['growth_rate_override'] = global_growth

# 5. --- THE CALCULATION ENGINE ---
# Calculate projected members for each chapter
chapters_df['2027_projected_members'] = chapters_df['2025_membership'] * (1 + chapters_df['growth_rate_override'])
# Calculate total national projected members
total_projected_members = chapters_df['2027_projected_members'].sum()
# Calculate each chapter's share of the total and thus its delegate estimate
chapters_df['2027_estimated_delegates'] = (chapters_df['2027_projected_members'] / total_projected_members) * (total_projected_members / 60)
# Simplify the calculation as we discussed
chapters_df['2027_estimated_delegates'] = chapters_df['2027_projected_members'] / 60
# Round the delegates to a whole number for practicality
chapters_df['2027_estimated_delegates'] = chapters_df['2027_estimated_delegates'].round().astype(int)

# 6. Let the user SEE and EDIT the chapter-level growth rates and final delegates!
st.header("Chapter-Level Projections")
st.caption("Edit the 'growth_rate_override' or '2027_estimated_delegates' columns directly in the table below.")

# This is the magic: Streamlit's data_editor makes the table editable.
edited_chapters_df = st.data_editor(chapters_df, num_rows="dynamic", use_container_width=True)

# 7. Show the results
st.subheader("Adjusted Chapter Summary")
st.write(edited_chapters_df[['chapter_name', '2025_membership', 'growth_rate_override', '2027_estimated_delegates']])

# 8. Button to export your worked-on data to a CSV
csv_chapters = edited_chapters_df.to_csv(index=False)
st.download_button("Download Chapter Data as CSV", data=csv_chapters, file_name="adjusted_chapters.csv", mime='text/csv')

ModuleNotFoundError: No module named 'streamlit'